# Activity 8

This activity tests your skills
in formulating an engineering problem as solving a system of linear algebraic equations,
as well as analyzing results thereof.

## Scenario

The lateral view of an [inclined strut](./act-08.png)
is designed as a ten-member truss.

You are tasked to analyze a scenario wherein
loads are approximated as concentrated forces 
acting on the non-support joints.
Members $AC$, $CE$, and $EG$ are collinear,
and perpendicular to the lines of action of loads $p_{1}$, $p_{2}$, and $p_{3}$.
Members $BD$, $DF$, and $FG$ are collinear,
and perpendicular
not just to the lines of action of loads $p_{4}$ and $p_{5}$,
but also to the supporting surface.

Let the dimensions, material makeup, and weight of the members be negligible.
Assume that the joints do not resist moments.
Determine the member forces developed
under such a conservative idealization of the strut.

## Modelling

Cast the original task as solving a system of linear algebraic equations
for the forces in $AC$, $BC$, $BD$, $CD$, $CE$, $DE$, $DF$, $EF$, $EG$, and $FG$.

Arrange the equations in the matrix-vector form
$\boldsymbol{K} \boldsymbol{\phi} = \boldsymbol{\delta}$,
where
$\boldsymbol{\phi}$ collects the member forces in the order presented above.
For uniformity, assume that tensile forces are positive.

## Core code

In [1]:
import math as mt

import numpy as np

Define a Python function
`get_coeffs_rhs()`
that takes four positional arguments
(namely, `d`, `h`, `theta`, `p1`, `p2`, `p3`, `p4`, `p5`,
in that order)
and returns two NumPy arrays
(namely,
`coeff_matrix`, `rhs_vector`
in that order).
The roles of these inputs and outputs should be evident.

In [2]:
def get_coeffs_rhs(d, h, theta, p1, p2, p3, p4, p5):
    # 1. Define Coordinates (Local Frame: B at origin)
    coords = {
        'A': (0, h),
        'B': (0, 0),
        'C': (d, 2 * h / 3),
        'D': (d, 0),
        'E': (2 * d, h / 3),
        'F': (2 * d, 0),
        'G': (3 * d, 0)
    }

    # 2. Define Members
    # AC, BC, BD, CD, CE, DE, DF, EF, EG, FG
    member_defs = [
        ('A', 'C'), ('B', 'C'), ('B', 'D'), ('C', 'D'), ('C', 'E'),
        ('D', 'E'), ('D', 'F'), ('E', 'F'), ('E', 'G'), ('F', 'G')
    ]
    
    # 3. Define Equation Mapping
    # 5 Free nodes = 10 equations.
    # Mapping: C -> rows 0,1; E -> 2,3; G -> 4,5; D -> 6,7; F -> 8,9
    node_map = {'C': 0, 'E': 2, 'G': 4, 'D': 6, 'F': 8}
    
    # Initialize K (10x10) and RHS (10)
    coeff_matrix = np.zeros((10, 10))
    rhs_vector = np.zeros(10)
    
    # 4. Build Coefficient Matrix
    for col_idx, (start_node, end_node) in enumerate(member_defs):
        xs, ys = coords[start_node]
        xe, ye = coords[end_node]
        
        length = mt.sqrt((xe - xs)**2 + (ye - ys)**2)
        cx = (xe - xs) / length
        cy = (ye - ys) / length
        
        # Add to start node equations
        if start_node in node_map:
            r = node_map[start_node]
            coeff_matrix[r, col_idx] = cx     # X-eq
            coeff_matrix[r + 1, col_idx] = cy # Y-eq
            
        # Add to end node equations (opposite direction)
        if end_node in node_map:
            r = node_map[end_node]
            coeff_matrix[r, col_idx] = -cx
            coeff_matrix[r + 1, col_idx] = -cy

    # 5. Build RHS Vecto
    # Calculate geometric properties for load projection
    L_top = mt.sqrt((3*d)**2 + h**2)
    sin_alpha = h / L_top
    cos_alpha = 3*d / L_top

    # Top Loads (p1@C, p2@E, p3@G)
    for node, p_val in [('C', p1), ('E', p2), ('G', p3)]:
        r = node_map[node]
        rhs_vector[r]     += p_val * sin_alpha
        rhs_vector[r + 1] += p_val * cos_alpha
        
    # Bottom Loads (p5@D, p4@F) - Note p5 is at D, p4 is at F
    # Force vector is P * (0, -1)
    # RHS = -Force = (0, P)
    rhs_vector[node_map['D'] + 1] += p5
    rhs_vector[node_map['F'] + 1] += p4
    
    return coeff_matrix, rhs_vector

D = 1.0
H = 6.0
THETA = 15 * (mt.pi / 180) # Convert degrees to radians (though not used for member forces)
P1 = 1000
P2 = 9000
P3 = 8000
P4 = 7000
P5 = 5000

## Assessment

Define Python variables
`D`, `H`, `THETA`, `P1`, `P2`, `P3`, `P4`, and `P5`
for storing the values of parameters
$d$, $h$, $\theta$, $p_{1}$, $p_{2}$, $p_{3}$, $p_{4}$, and $p_{5}$,
respectively.

Every time you solve for $\boldsymbol{\phi}$,
- use `get_coeffs_rhs()` to generate the coefficient matrix and the right-hand-side vector,
  respectively storing them in Python variables `K` and `d`;
- show the determinant and the condition number of the coefficient matrix;
  and
- print $\boldsymbol{\phi}$.


### Task 1

Determine the member forces when $C$ is loaded by a unit force and all other joints are unloaded.

Define a Python variable `phi_1` to store $\boldsymbol{\phi}$ for this task.

In [3]:
K, d = get_coeffs_rhs(D, H, THETA, 1, 0, 0, 0, 0)
print(f"Determinant: {np.linalg.det(K):.4e}")
print(f"Condition Number: {np.linalg.cond(K):.4e}")

phi_1 = np.linalg.solve(K, d)
print("phi_1:", phi_1)

Determinant: 4.6567e-01
Condition Number: 6.5426e+00
phi_1: [-1.16666667 -1.53659074 -0.          0.          0.         -0.
 -0.          0.          0.         -0.        ]


### Task 2

Determine the member forces when $E$ is loaded by a unit force and all other joints are unloaded.

Define a Python variable `phi_2` to store $\boldsymbol{\phi}$ for this task.

In [4]:
K, d = get_coeffs_rhs(D, H, THETA, 0, 1, 0, 0, 0)
print(f"Determinant: {np.linalg.det(K):.4e}")
print(f"Condition Number: {np.linalg.cond(K):.4e}")

phi_2 = np.linalg.solve(K, d)
print("phi_2:", phi_2)

Determinant: 4.6567e-01
Condition Number: 6.5426e+00
phi_2: [-0.33333333 -0.76829537 -0.55901699  1.11803399 -0.75       -1.25
 -0.          0.          0.         -0.        ]


### Task 3

Determine the member forces when $G$ is loaded by a unit force and all other joints are unloaded.

Define a Python variable `phi_3` to store $\boldsymbol{\phi}$ for this task.

In [5]:
K, d = get_coeffs_rhs(D, H, THETA, 0, 0, 1, 0, 0)
print(f"Determinant: {np.linalg.det(K):.4e}")
print(f"Condition Number: {np.linalg.cond(K):.4e}")

phi_3 = np.linalg.solve(K, d)
print("phi_3:", phi_3)

Determinant: 4.6567e-01
Condition Number: 6.5426e+00
phi_3: [ 0.5        -0.         -1.11803399  0.          0.5        -0.
 -1.11803399  0.          0.5        -1.11803399]


### Task 4

Determine the member forces when $F$ is loaded by a unit force and all other joints are unloaded.

Define a Python variable `phi_4` to store $\boldsymbol{\phi}$ for this task.

In [6]:
K, d = get_coeffs_rhs(D, H, THETA, 0, 0, 0, 1, 0)
print(f"Determinant: {np.linalg.det(K):.4e}")
print(f"Condition Number: {np.linalg.cond(K):.4e}")

phi_4 = np.linalg.solve(K, d)
print("phi_4:", phi_4)

Determinant: 4.6567e-01
Condition Number: 6.5426e+00
phi_4: [ 0.74535599 -0.34359214 -0.25        0.5         0.55901699 -0.55901699
 -0.          1.          0.         -0.        ]


### Task 5

Determine the member forces when $D$ is loaded by a unit force and all other joints are unloaded.

Define a Python variable `phi_5` to store $\boldsymbol{\phi}$ for this task.

In [7]:
K, d = get_coeffs_rhs(D, H, THETA, 0, 0, 0, 0, 1)
print(f"Determinant: {np.linalg.det(K):.4e}")
print(f"Condition Number: {np.linalg.cond(K):.4e}")

phi_5 = np.linalg.solve(K, d)
print("phi_5:", phi_5)

Determinant: 4.6567e-01
Condition Number: 6.5426e+00
phi_5: [ 0.372678   -0.68718427 -0.          1.          0.         -0.
 -0.          0.          0.         -0.        ]


### Task 6

Determine the member forces
when $C$, $E$, $G$, $F$, and $D$
are loaded by unit forces.

Define a Python variable `phi_6` to store $\boldsymbol{\phi}$ for this task.

In [8]:
K, d = get_coeffs_rhs(D, H, THETA, 1, 1, 1, 1, 1)
print(f"Determinant: {np.linalg.det(K):.4e}")
print(f"Condition Number: {np.linalg.cond(K):.4e}")

phi_6 = np.linalg.solve(K, d)
print("phi_6:", phi_6)

Determinant: 4.6567e-01
Condition Number: 6.5426e+00
phi_6: [ 0.11803399 -3.33566252 -1.92705098  2.61803399  0.30901699 -1.80901699
 -1.11803399  1.          0.5        -1.11803399]


### Task 7

Similar to Task 6,
determine the member forces
when $C$, $E$, $G$, $F$, and $D$
are loaded by unit forces,
but use the results in Tasks 1 - 5.

Define a Python variable `phi_7` to store $\boldsymbol{\phi}$ for this task.

In [9]:
phi_7 = phi_1 + phi_2 + phi_3 + phi_4 + phi_5
print("phi_7:", phi_7)

phi_7: [ 0.11803399 -3.33566252 -1.92705098  2.61803399  0.30901699 -1.80901699
 -1.11803399  1.          0.5        -1.11803399]


### Task 8

Determine the member forces using the assigned values of
$p_{1}$, $p_{2}$, $p_{3}$, $p_{4}$, and $p_{5}$.

Define a Python variable `phi_8` to store $\boldsymbol{\phi}$ for this task.

In [10]:
K, d = get_coeffs_rhs(D, H, THETA, P1, P2, P3, P4, P5)
print(f"Determinant: {np.linalg.det(K):.4e}")
print(f"Condition Number: {np.linalg.cond(K):.4e}")

phi_8 = np.linalg.solve(K, d)
print("phi_8:", phi_8)

Determinant: 4.6567e-01
Condition Number: 6.5426e+00
phi_8: [  6914.21526208 -14292.31538881 -15725.42485937  18562.30589875
   1163.11896062 -15163.11896062  -8944.27191      7000.
   4000.          -8944.27191   ]


### Task 9

Similar to Task 8,
determine the member forces using the assigned values of
$p_{1}$, $p_{2}$, $p_{3}$, $p_{4}$, and $p_{5}$,
but use the results in Tasks 1 - 5.


Define a Python variable `phi_9` to store $\boldsymbol{\phi}$ for this task.

In [11]:
phi_9 = (P1 * phi_1) + (P2 * phi_2) + (P3 * phi_3) + (P4 * phi_4) + (P5 * phi_5)
print("phi_9:", phi_9)

phi_9: [  6914.21526208 -14292.31538881 -15725.42485937  18562.30589875
   1163.11896062 -15163.11896062  -8944.27191      7000.
   4000.          -8944.27191   ]


### Task 10

Determine the magnitudes of the support reactions
using the assigned values of
$p_{1}$, $p_{2}$, $p_{3}$, $p_{4}$, and $p_{5}$.

Define Python variables `rxn_A` and `rxn_B` 
to store the forces developed at $A$ and $B$, respectively.

In [12]:
# Reaction at A
# Magnitude of reaction = Magnitude of force in AC.
rxn_A = abs(phi_8[0]) # index 0 is AC

# Reaction at B
# Vector B->C: (D, 2H/3)
vec_BC = np.array([D, 2 * H / 3])
len_BC = np.linalg.norm(vec_BC)
u_BC = vec_BC / len_BC

# Vector B->D: (D, 0)
vec_BD = np.array([D, 0])
len_BD = np.linalg.norm(vec_BD)
u_BD = vec_BD / len_BD

# Force vectors acting ON B (Tension pulls away from B, along u_BC/u_BD)
force_BC_vec = phi_8[1] * u_BC
force_BD_vec = phi_8[2] * u_BD

# Reaction balances these: R_B + F_BC + F_BD = 0 => R_B = -(F_BC + F_BD)
R_B_vec = -(force_BC_vec + force_BD_vec)
rxn_B = np.linalg.norm(R_B_vec)

print(f"Reaction at A: {rxn_A:.4f}")
print(f"Reaction at B: {rxn_B:.4f}")

Reaction at A: 6914.2153
Reaction at B: 23676.5782


## Instructions

Do not use any library or module other than those in the Imports section.

### Parameters

For each set,
the values shown are that of
$d$, $h$, $\theta$, $p_{1}$, $p_{2}$, $p_{3}$, $p_{4}$, and $p_{5}$,
respectively.

- Set 1: 1.5 ft, 4.5 ft, 21°, 5 kN, 7 kN, 6 kN, 8 kN, 1 kN
- Set 2: 2.0 ft, 3.0 ft, 17°, 9 kN, 6 kN, 5 kN, 1 kN, 7 kN
- Set 3: 1.0 ft, 6.0 ft, 15°, 1 kN, 9 kN, 8 kN, 7 kN, 5 kN

### Scoring

In each of Tasks 1 - 9,
obtaining a correct member force merits one (1) point.

In each Task but 7, 9, and 10,
- obtaining a correct determinant merits one (1) point,
  and
- obtaining a correct condition number merits one (1) point.

For Task 10,
obtaining a correct reaction force merits five (5) points.

Every violation of an instruction
means a deduction of one (1) point.

All in all, one may earn up to 124 points for this activity.

### Submission

Download this notebook file,
and save with a filename following the pattern
`ACT-08_<Group name>`,
where the group name is as listed in class.
For example, if you belong to the group SixIsEven,
then your notebook should be named `ACT-08_SixIsEven.ipynb`.
Submit your notebook via the classwork platform for this activity in Google Classroom.
Submissions beyond the deadline will not be considered.

Lastly, the use of AI tools to answer this exam is not prohibited,
but it is of ethical interest to disclose such use.
This is in line with the
[MSU Policy on the Fair and Ethical Use of AI and Its Applications](https://www.msumain.edu.ph/wp-content/uploads/2024/05/MSU-Policy-on-Ethical-use-of-AI-Policies.pdf).
As such, please include a brief statement (in a private comment to this classwork)
declaring which and how AI tools are used in your work.

*Last updated by Christian Cahig on 2025-11-25*